In [ ]:
#!pip install aquacrop==2.2
#!pip install numba==0.55
#!pip install statsmodels==0.13.2
#!pip install SALib
#!pip install -U scikit-learn scipy matplotlib
!pip install --upgrade pyswarm  #https://pythonhosted.org/pyswarm/

In [2]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn.metrics as metrics
from pyswarm import pso
import os
from os import chdir, getcwd


In [3]:
os.chdir('/home/jupyter-wndlovu/') # change working directory

wd=getcwd()

In [83]:
def soilCompart(soilList):
    custom_soil = []
    for df in soilList:

        #soils_df = df.groupby(['Year'])
        soils_df = [v for k, v in df.groupby('Year')]
        #print(soils_df)
        for j in soils_df:
            #df_list = []
            soils_df_pivot = j.drop(['crop_mn_codeyear', 'Year'], axis=1)
            #print(soils_df_pivot)
            for i, row in soils_df_pivot.iterrows():
                #print(row)
                #print(soils_df_pivot)
                if row['depth'] == "0_5":
                    pred_thWP_5 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_5 = pred_thWP_5 + (0.14 * pred_thWP_5) - 0.02
                    pred_thFC_5 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_5 = pred_thFC_5 + (1.283 * (np.power(pred_thFC_5, 2))) - (0.374 * pred_thFC_5) - 0.015
                        #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_5 =soils_df_pivot["theta_s"][i]
                    ks_5=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "100_200":
                    pred_thWP_200 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_200 = pred_thWP_200 + (0.14 * pred_thWP_200) - 0.02
                    pred_thFC_200 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_200 = pred_thFC_200 + (1.283 * (np.power(pred_thFC_200, 2))) - (0.374 * pred_thFC_200) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_200 =soils_df_pivot["theta_s"][i]
                    ks_200=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "15_30":
                    pred_thWP_30 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_30 = pred_thWP_30 + (0.14 * pred_thWP_30) - 0.02
                    pred_thFC_30 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_30 = pred_thFC_30 + (1.283 * (np.power(pred_thFC_30, 2))) - (0.374 * pred_thFC_30) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_30 =soils_df_pivot["theta_s"][i]
                    ks_30=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "30_60":
                    pred_thWP_60 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_60 = pred_thWP_60 + (0.14 * pred_thWP_60) - 0.02
                    pred_thFC_60 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_60 = pred_thFC_60 + (1.283 * (np.power(pred_thFC_60, 2))) - (0.374 * pred_thFC_60) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_60 =soils_df_pivot["theta_s"][i]
                    ks_60=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "5_15":
                    pred_thWP_15 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_15 = pred_thWP_15 + (0.14 * pred_thWP_15) - 0.02
                    pred_thFC_15 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_15 = pred_thFC_15 + (1.283 * (np.power(pred_thFC_15, 2))) - (0.374 * pred_thFC_15) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_15 =soils_df_pivot["theta_s"][i]
                    #print(ts_15)
                    ks_15=(soils_df_pivot['ksat'][i])*240
                if row['depth'] == "60_100":
                    pred_thWP_100 = ((-0.024*((soils_df_pivot['sand'][i])/100))) + ((0.487*((soils_df_pivot['clay'][i])/100))) + ((0.006*((soils_df_pivot['om'][i])/100))) + ((0.005*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.013*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.068*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.031
                    wp_100 = pred_thWP_100 + (0.14 * pred_thWP_100) - 0.02
                    pred_thFC_100 = ((-0.251*((soils_df_pivot['sand'][i])/100))) + ((0.195*((soils_df_pivot['clay'][i])/100)))+ ((0.011*((soils_df_pivot['om'][i])/100))) + ((0.006*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['om'][i])/100))- ((0.027*((soils_df_pivot['clay'][i])/100))*((soils_df_pivot['om'][i])/100))+ ((0.452*((soils_df_pivot['sand'][i])/100))*((soils_df_pivot['clay'][i])/100))+ 0.299
                    fc_100 = pred_thFC_100 + (1.283 * (np.power(pred_thFC_100, 2))) - (0.374 * pred_thFC_100) - 0.015
                    #fc = pred_thFC + (1.283 * (pred_thFC*pred_thFC)) - (0.374 * pred_thFC) - 0.015
                    ts_100 =soils_df_pivot["theta_s"][i]
                    ks_100=(soils_df_pivot['ksat'][i])*240



                    custom = Soil('custom',cn=46,rew=7, dz=[0.025]*2+[0.05]*2+[0.075]*2+[0.15]*2+[0.2]*2+[0.5]*2)

                    custom.add_layer(thickness=0.05,thS=ts_5, # assuming soil properties are the same in the upper 0.1m
                         Ksat=ks_5,thWP =wp_5 , 
                         thFC = fc_5, penetrability = 100.0)
                    custom.add_layer(thickness=0.15,thS=ts_15, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_15,thWP =wp_15 , 
                                 thFC = fc_15, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_30, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_30,thWP =wp_30 , 
                                 thFC = fc_30, penetrability = 100.0)
                    custom.add_layer(thickness=0.3,thS=ts_60, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_60,thWP =wp_60 , 
                                 thFC = fc_60, penetrability = 100.0)
                    custom.add_layer(thickness=0.4,thS=ts_100, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_100,thWP =wp_100 , 
                                 thFC = fc_100, penetrability = 100.0)
                    custom.add_layer(thickness=1,thS=ts_200, # assuming soil properties are the same in the upper 0.1m
                                 Ksat=ks_200,thWP =wp_200 , 
                                 thFC = fc_200, penetrability = 100.0)
                    custom_soil.append(custom)
                    #df_list.append(custom)

                    #print(custom)

    return(custom_soil)

In [150]:
with open(wd + '/eggs/data/input_dict.pickle', 'rb') as input_data: 
    input_dict = pickle.load(input_data) 

## Irrigated corn

In [151]:
irrig_corn = '1' # code for irrigated corn
irrig_corn_dict = {k:v for (k,v) in input_dict.items() if irrig_corn in k}

In [152]:
# calibrate using the decatur, gove, graham, logan, rawlins
calibration = list(irrig_corn_dict.items())[:5]
#calibration

In [153]:
#calibration[0][1]

In [154]:
# getting the different datasets
gridmet = [sublist[1][0] for sublist in calibration] # data stored in nested list with 0 having the 
lai = [sublist[1][1] for sublist in calibration]
soil = [sublist[1][2] for sublist in calibration]

### Soil data is currently being saved in one list - wondering if saving it in 5 lists make more sense (one list for each county)

In [116]:
soil_irrig = soilCompart(soil)

In [ ]:
dec_met = gridmet[0]
dec_met

In [ ]:
gridmet_df = [v for k, v in dec_met.groupby('Year')]

for met in gridmet_df:
    yr = met.Year.unique()
    print(yr)

In [159]:
calib_counties = []
for county in gridmet:
    model_results = []
    gridmet_df = [v for k, v in dec_met.groupby('Year')]
    for met in gridmet_df:
        yr = met['Year'].unique()[0]
        wdf = met[['MinTemp', 
                             'MaxTemp',
                             'Precipitation', 
                             'ReferenceET',
                             'Date'
                            ]]
        sim_start = f'{yr}/01/01' #dates to match crop data
        sim_end = f'{yr}/12/31'
        #print(sim_start)
        initWC = InitialWaterContent(value=['FC'])
        irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation

        # uncalibrated model - yield
        crop_uncalibrated = Crop('Maize', planting_date='05/01') 

        # run model
        model_uc = AquaCropModel(sim_start,sim_end,wdf,soil_irrig[0],crop_uncalibrated,initWC, irr_mngt)
        model_uc.run_model(till_termination=True) # run model till the end
        #model_uc_et = model_uc._outputs.water_flux
        model_uc_irr = model_uc._outputs.final_stats
        model_results.append(model_uc_irr)

    model_results_df = pd.concat(model_results)
    
    




In [161]:
model_results_df

,Season,crop Type,Harvest Date (YYYY/MM/DD),Harvest Date (Step),Yield (tonne/ha),Seasonal irrigation (mm)
0,0,Maize,2006-09-10,251,14.160832,701.290195
0,0,Maize,2007-09-10,251,14.317896,653.414124
0,0,Maize,2008-09-10,252,14.235415,617.066997
0,0,Maize,2009-09-10,251,14.005505,506.783554
0,0,Maize,2010-09-10,251,14.440951,604.040825
0,0,Maize,2011-09-10,251,14.424683,665.534344
0,0,Maize,2012-09-10,252,13.675188,963.896213
0,0,Maize,2013-09-10,251,13.751165,739.243477
0,0,Maize,2014-09-10,251,14.302506,632.965839
0,0,Maize,2015-09-10,251,14.124127,554.933543


In [ ]:

sim_start = f'{1979}/01/01' #dates to match crop data
sim_end = f'{1979}/12/31'
initWC = InitialWaterContent(value=['FC'])
irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation


# uncalibrated model - yield
crop_uncalibrated = Crop('Maize', planting_date='05/01') 

# run model
model_uc = AquaCropModel(sim_start,sim_end,wdf,custom,crop_uncalibrated,initWC, irr_mngt)
model_uc.run_model(till_termination=True) # run model till the end
#model_uc_et = model_uc._outputs.water_flux
model_uc_irr = model_uc._outputs.final_stats
#model_df_water_storage = model._outputs.water_storage
#model_df_crp_grwth = model._outputs.crop_growth


In [ ]:
sim_start = '2000/01/01' #dates to match crop data
sim_end = '2020/12/31'
initWC = InitialWaterContent(value=['FC'])
irr_mngt = IrrigationManagement(irrigation_method=1,SMT=[80]*4) # no irrigation


# uncalibrated model - yield
crop_uncalibrated = Crop('Maize', planting_date='05/01') 

# run model
model_uc = AquaCropModel(sim_start,sim_end,wdf,custom,crop_uncalibrated,initWC, irr_mngt)
model_uc.run_model(till_termination=True) # run model till the end
#model_uc_et = model_uc._outputs.water_flux
model_uc_irr = model_uc._outputs.final_stats
#model_df_water_storage = model._outputs.water_storage
#model_df_crp_grwth = model._outputs.crop_growth

## Create year variable
model_uc_irr = model_uc_irr.assign(Year =  model_uc_irr['Harvest Date (YYYY/MM/DD)'].dt.year)
model_uc_irr = model_uc_irr.rename(columns={
                   'Yield (tonne/ha)': 'Uncalib Yield (t/ha)',
                   'Seasonal irrigation (mm)': 'Uncalib Irrigation (mm)'
                   })
model_uc_irr = model_uc_irr[['Year', 'Uncalib Yield (t/ha)', 'Uncalib Irrigation (mm)']]
